# Contents
1. Word Vectors
2. Word-Document Matrix
3. Window based Co-occurence Matrix
4. Matrix Decompostion using SVD
5. Clustering the points using spherical k-means

## 1. Representing Words as One-hot vectors

The original document of the `wiki.dl.history.txt` is [this]('wiki.dl.history.txt')

In [1]:
# open the text file to process
with open('wiki.dl.history.txt') as f:
    corpus_raw = f.read()
print(corpus_raw[:1000])    

According to a survey,[4] the expression Deep Learning was introduced to the Machine Learning community by Rina Dechter in 1986,[22] and later to Artificial Neural Networks by Igor Aizenberg and colleagues in 2000, in the context of Boolean threshold neurons.[23] In 2005, Faustino Gomez and Jürgen Schmidhuber published a paper on learning deep POMDPs[24] through neural networks for reinforcement learning. In 2006, a publication by Hinton, Osindero, and Teh[25][26] drew attention by showing how many-layered feedforward neural network could be effectively pre-trained one layer at a time, treating each layer in turn as an unsupervised restricted Boltzmann machine, then fine-tuning it using supervised backpropagation.[27] The paper referred to learning for deep belief nets. A Google Ngram chart shows that the usage of the term has taken off since 2000.[28] The underlying concepts and many techniques, however, date to earlier decades.

The first general, working learning algorithm for super

First, remove the special patterns like `[Number]`(e.g. `[4]`) so that we can extract only the pure words.

In [2]:
import re
corpus = re.sub('\[[0-9]*\]','',corpus_raw)
print(corpus[:1000])

According to a survey, the expression Deep Learning was introduced to the Machine Learning community by Rina Dechter in 1986, and later to Artificial Neural Networks by Igor Aizenberg and colleagues in 2000, in the context of Boolean threshold neurons. In 2005, Faustino Gomez and Jürgen Schmidhuber published a paper on learning deep POMDPs through neural networks for reinforcement learning. In 2006, a publication by Hinton, Osindero, and Teh drew attention by showing how many-layered feedforward neural network could be effectively pre-trained one layer at a time, treating each layer in turn as an unsupervised restricted Boltzmann machine, then fine-tuning it using supervised backpropagation. The paper referred to learning for deep belief nets. A Google Ngram chart shows that the usage of the term has taken off since 2000. The underlying concepts and many techniques, however, date to earlier decades.

The first general, working learning algorithm for supervised, deep, feedforward, multi

In [3]:
# separate the corpus into words
corpus_list = corpus.lower().split()
corpus_list[:10]

['according',
 'to',
 'a',
 'survey,',
 'the',
 'expression',
 'deep',
 'learning',
 'was',
 'introduced']

In [4]:
# Build the Vocabulary
vocabulary = list({word for word in corpus_list})
n_words = len(vocabulary)
print('The number of words in the vocabulary: %i' % n_words)

The number of words in the vocabulary: 654


In [5]:
# Build the one-hot vectors for the vocabulary
one_hot = dict()
for i,word in enumerate(vocabulary):
    one_hot[word] = [1 if i==j else 0 for j in range(n_words)]

In [6]:
# One-hot representations of a document
corpus2id = [one_hot[word] for word in corpus_list]
print(corpus2id[:3])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 2. Word-Document Matrix

In [7]:
from collections import Counter
counts = Counter(corpus_list).most_common()

# Remove stopwords
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
counts = [(word, count) for (word,count) in counts if word not in stops]

# Choose only top-V words from the document as a Vocabulary
V = 50
counts = counts[:V]
print(counts)

[('deep', 27), ('neural', 23), ('speech', 18), ('learning', 15), ('recognition', 12), ('networks', 10), ('network', 8), ('published', 6), ('used', 6), ('learning.', 5), ('using', 5), ('training', 5), ('generative', 5), ('recognition.', 5), ('layer', 4), ('unsupervised', 4), ('google', 4), ('many', 4), ('layers', 4), ('computer', 4), ('systems', 4), ('3-d', 4), ('object', 4), ('model', 4), ('cresceptron', 4), ('models', 4), ('speaker', 4), ('machine', 3), ('later', 3), ('colleagues', 3), ('schmidhuber', 3), ('paper', 3), ('feedforward', 3), ('could', 3), ('one', 3), ('supervised', 3), ('however,', 3), ('first', 3), ('algorithm', 3), ('method', 3), ('data', 3), ('called', 3), ('demonstrated', 3), ('et', 3), ('al.', 3), ('recognizing', 3), ('days.', 3), ('brain', 3), ('pre-training', 3), ('recurrent', 3)]


In [8]:
# Word-ID Mapping
word2id = {tup[0]: i for i,tup in enumerate(counts)}
id2word = {i: tup[0] for i,tup in enumerate(counts)}
print(word2id)

{'deep': 0, 'neural': 1, 'speech': 2, 'learning': 3, 'recognition': 4, 'networks': 5, 'network': 6, 'published': 7, 'used': 8, 'learning.': 9, 'using': 10, 'training': 11, 'generative': 12, 'recognition.': 13, 'layer': 14, 'unsupervised': 15, 'google': 16, 'many': 17, 'layers': 18, 'computer': 19, 'systems': 20, '3-d': 21, 'object': 22, 'model': 23, 'cresceptron': 24, 'models': 25, 'speaker': 26, 'machine': 27, 'later': 28, 'colleagues': 29, 'schmidhuber': 30, 'paper': 31, 'feedforward': 32, 'could': 33, 'one': 34, 'supervised': 35, 'however,': 36, 'first': 37, 'algorithm': 38, 'method': 39, 'data': 40, 'called': 41, 'demonstrated': 42, 'et': 43, 'al.': 44, 'recognizing': 45, 'days.': 46, 'brain': 47, 'pre-training': 48, 'recurrent': 49}


In [9]:
# Obtain the word-count vector representation of a document
import numpy as np
doc1 = np.zeros((V,1), dtype=int)
for i, tup in enumerate(counts):
    count = tup[1]
    doc1[i,:] = count
print(doc1)

[[27]
 [23]
 [18]
 [15]
 [12]
 [10]
 [ 8]
 [ 6]
 [ 6]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 5]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 4]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]
 [ 3]]


the origial document for `wiki.nlp.history` is [here](https://en.wikipedia.org/wiki/Natural_language_processing#History)

In [10]:
with open('wiki.nlp.history.txt') as f:
    corpus_ = re.sub('\[[0-9]*\]','',f.read())
corpus_nlp = corpus_.lower().split()
doc2 = np.zeros((V,1), int)
for word in corpus_nlp:
    if word in word2id:
        doc2[word2id[word],:] += 1
print(doc2)

[[1]
 [0]
 [1]
 [5]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [2]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [6]
 [0]
 [0]
 [8]
 [0]
 [0]
 [0]
 [0]
 [4]
 [0]
 [7]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [5]
 [1]
 [0]
 [0]
 [2]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [11]:
X = np.hstack((doc1,doc2))
X

array([[27,  1],
       [23,  0],
       [18,  1],
       [15,  5],
       [12,  1],
       [10,  0],
       [ 8,  0],
       [ 6,  1],
       [ 6,  0],
       [ 5,  0],
       [ 5,  2],
       [ 5,  0],
       [ 5,  0],
       [ 5,  0],
       [ 4,  0],
       [ 4,  1],
       [ 4,  0],
       [ 4,  6],
       [ 4,  0],
       [ 4,  0],
       [ 4,  8],
       [ 4,  0],
       [ 4,  0],
       [ 4,  0],
       [ 4,  0],
       [ 4,  4],
       [ 4,  0],
       [ 3,  7],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  1],
       [ 3,  5],
       [ 3,  1],
       [ 3,  0],
       [ 3,  0],
       [ 3,  2],
       [ 3,  1],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0],
       [ 3,  0]])

## 3. Window based Co-occurence Matrix

In [12]:
# Define helper functions

def window_generator(corpus_list, window_size):
    '''Generates the window of window_size using the words in the corpus.
    
    Args:
        corpus_list: list of word strings in the corpus. e.g. ['hello','my','name','is']
        window_size: an integer that indicates the size of the window
        
    Returns:
        window: list of word strings, which is a subset of the corpus.
    '''
    corpus_len = len(corpus_list)
    for i in range(corpus_len):
        # left end(inclusive)
        left_end = max(0, i - window_size)
        # right end(inclusive)
        right_end = min(i + window_size, corpus_len - 1)
        window = corpus_list[left_end:right_end+1]
        yield window
        
def count_cooc(window):
    '''Count the co-occurences of words within the specified window, and updates the affinity matrix
    
    Args:
        window: list of word strings, which is a subset of the corpus. e.g. ['hello','my','name','is']
        
    Returns:
        None
    '''
    queue = [word for word in window if word in word2id]
    while len(queue) >= 2:
        this_word = queue[0] # current word to process
        del queue[0]
        for other_word in queue:
            # base case: same word is repeated
            if this_word == other_word:
                continue
            # usual case: update the co-occurence matrix
            X[word2id[this_word], word2id[other_word]] += 1
            X[word2id[other_word], word2id[this_word]] += 1

In [13]:
X = np.zeros((V,V), dtype=int) # affinity matrix
window_size = 1
# Generate the windows and count the co-occurences
for window in window_generator(corpus_list, window_size):
    count_cooc(window)
    
# Check which words co-occurred
it = np.nditer(X, flags=['multi_index'])
while not it.finished:
    if it[0] != 0:
        i,j = it.multi_index
        if i > j:
            print(id2word[i],id2word[j])
    it.iternext()

neural deep
learning deep
learning speech
recognition speech
networks deep
networks neural
networks speech
network deep
network neural
published neural
published network
learning. deep
generative deep
generative speech
generative using
recognition. speech
unsupervised used
google used
many speech
many recognition.
systems used
3-d recognition
object recognition
object 3-d
model 3-d
model object
cresceptron used
models deep
models speech
models using
models generative
speaker speech
speaker recognition
speaker networks
speaker recognition.
machine learning
machine learning.
later published
schmidhuber published
schmidhuber used
schmidhuber unsupervised
paper learning
paper published
feedforward deep
feedforward neural
feedforward networks
feedforward network
could neural
could network
one speech
one recognition.
one layer
supervised deep
supervised learning.
supervised using
however, neural
however, training
however, many
first used
algorithm learning
method deep
method learning
data tr

## 4. Matrix Decomposition using SVD

Default SVD

In [14]:
from numpy.linalg import svd
print(X.shape)

# Default SVD
U, s, V_T = svd(X)
S = np.diag(s)
V = V_T.T
print(U.shape,S.shape,V.shape)
print("Preserved variance: %.4f" % 1)

(50, 50)
(50, 50) (50, 50) (50, 50)
Preserved variance: 1.0000


Low-rank approximation using the Truncated SVD

In [15]:
k = 10
U_k = U[:,:k]
s_k = s[:k]
S_k = np.diag(s_k)
V_k = V[:,:k]
print(U_k.shape,S_k.shape,V_k.shape)
print("Preserved variance: %.4f" % ((s_k**2).sum()/(s**2).sum()))
X_k_np = np.dot(np.dot(U_k,S_k),V_k.T)

(50, 10) (10, 10) (50, 10)
Preserved variance: 0.9136


In [16]:
X - X_k_np

array([[  7.79459557e-03,  -4.60781844e-03,   3.85495896e-03, ...,
         -1.21302405e-03,   1.56091796e-02,  -7.82903066e-03],
       [ -4.60781844e-03,   1.05034395e-02,  -6.52047100e-03, ...,
         -9.38019056e-04,  -4.01760331e-03,   3.22364157e-02],
       [  3.85495896e-03,  -6.52047100e-03,  -3.07212427e-02, ...,
          9.35489148e-03,   7.02801345e-05,   4.89849804e-03],
       ..., 
       [ -1.21302405e-03,  -9.38019056e-04,   9.35489148e-03, ...,
         -2.49822036e-03,  -8.43468738e-03,   1.31145968e-04],
       [  1.56091796e-02,  -4.01760331e-03,   7.02801345e-05, ...,
         -8.43468738e-03,  -2.26320948e-02,  -3.85254850e-02],
       [ -7.82903066e-03,   3.22364157e-02,   4.89849804e-03, ...,
          1.31145968e-04,  -3.85254850e-02,  -4.27370242e-02]])

Since 91% of the variance is captured by the latent representation, the errors are often very small.

## 5. Clustering using spherical k-means

In [17]:
V_k_normalized = V_k.copy()
for row in range(V_k_normalized.shape[0]):
    v_k = V_k_normalized[row]
    norm = np.sqrt(np.dot(v_k,v_k))
    if norm == 0:
        continue
    V_k_normalized[row] = v_k/norm

In [18]:
from spherecluster import SphericalKMeans

# Run spherical K-means
K = 5
skm = SphericalKMeans(n_clusters=K, n_jobs=-1, random_state=0).fit(X)

# Print out the cluster members
for cluster in range(K):
    members = [id2word[i] for i,label in enumerate(skm.labels_) if label==cluster]
    print(members)

['recognition', 'used', 'using', 'training', 'generative', 'recognition.', 'layer', 'unsupervised', 'google', 'many', 'systems', '3-d', 'model', 'cresceptron', 'models', 'later', 'schmidhuber', 'one', 'supervised', 'first', 'data', 'et', 'al.', 'recognizing', 'days.', 'brain', 'pre-training']
['deep', 'machine', 'paper', 'algorithm', 'method', 'called', 'demonstrated']
['networks', 'network', 'published', 'feedforward', 'could', 'however,', 'recurrent']
['speech', 'object', 'speaker']
['neural', 'learning', 'learning.', 'layers', 'computer', 'colleagues']


We can see that some of the clusters in the 10-dimensional Latent Space are of good quality, even if the size of our data is very small.